In [67]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np
import json
from time import sleep, perf_counter as pf
import itertools as it

In [68]:
date = pd.DataFrame(pd.date_range('01/01/2015', '31/12/2017', freq='D'), columns=['Date'])
date.insert(0, 'Date_key', date.index)
display(date)

,Date_key,Date
0,0,2015-01-01
1,1,2015-01-02
2,2,2015-01-03
3,3,2015-01-04
4,4,2015-01-05
5,5,2015-01-06
6,6,2015-01-07
7,7,2015-01-08
8,8,2015-01-09
9,9,2015-01-10


In [69]:
# Add date information
date['Day_of_week'] = date.Date.dt.dayofweek
date['Year'] = date.Date.dt.year
date['Month'] = date.Date.dt.month
date['Day'] = date.Date.dt.day
date['Week'] = date.Date.dt.week
date['Weekend'] = 0
date.loc[
    (date.Day_of_week == 0) |
    (date.Day_of_week == 6),
    'Weekend'
] = 1
date['season'] = (date.Date.dt.month%12 + 3)//3
date.to_csv('datasets/final/date.csv', index=False)
display(date)

,Date_key,Date,Day_of_week,Year,Month,Day,Week,Weekend,season
0,0,2015-01-01,3,2015,1,1,1,0,1
1,1,2015-01-02,4,2015,1,2,1,0,1
2,2,2015-01-03,5,2015,1,3,1,0,1
3,3,2015-01-04,6,2015,1,4,1,1,1
4,4,2015-01-05,0,2015,1,5,2,1,1
5,5,2015-01-06,1,2015,1,6,2,0,1
6,6,2015-01-07,2,2015,1,7,2,0,1
7,7,2015-01-08,3,2015,1,8,2,0,1
8,8,2015-01-09,4,2015,1,9,2,0,1
9,9,2015-01-10,5,2015,1,10,2,0,1


In [70]:
# Add Crime
crime = pd.read_csv('datasets/crime_output.csv')
date.Date.astype(str)
crime['DATE'] = pd.to_datetime(crime['DATE'])
crime['start_dt'] = pd.to_datetime(crime['start_dt'])
crime['end_dt'] = pd.to_datetime(crime['end_dt'])
df = pd.merge(date, crime, left_on='Date', right_on='DATE', how='left')
display(df)

,Date_key,Date,Day_of_week,Year,Month,Day,Week,Weekend,season,Crime_key,...,Details,IS_FATAL,IS_TRAFFIC,LAT,LON,NEIGHBORHOOD,TIME,TYPE,end_dt,start_dt
0,0,2015-01-01,3,2015,1,1,1,0,1,440785,...,NaN,0,0,49.267152,-123.145645,Fairview,18:30:00,Theft from Vehicle,NaT,NaT
1,0,2015-01-01,3,2015,1,1,1,0,1,440797,...,NaN,0,0,49.280496,-123.109639,Central Business District,04:40:00,Theft from Vehicle,NaT,NaT
2,0,2015-01-01,3,2015,1,1,1,0,1,440798,...,NaN,0,0,49.280496,-123.109639,Central Business District,04:48:00,Theft from Vehicle,NaT,NaT
3,0,2015-01-01,3,2015,1,1,1,0,1,440802,...,NaN,0,0,49.280489,-123.109473,Central Business District,14:00:00,Theft from Vehicle,NaT,NaT
4,0,2015-01-01,3,2015,1,1,1,0,1,441402,...,NaN,0,0,49.271399,-123.046030,Hastings-Sunrise,02:00:00,Theft of Vehicle,NaT,NaT
5,0,2015-01-01,3,2015,1,1,1,0,1,442639,...,NaN,0,0,49.283632,-123.109384,Central Business District,03:01:00,Mischief,NaT,NaT
6,0,2015-01-01,3,2015,1,1,1,0,1,443601,...,NaN,0,0,49.283827,-123.024709,Hastings-Sunrise,20:05:00,Mischief,NaT,NaT
7,0,2015-01-01,3,2015,1,1,1,0,1,443739,...,NaN,0,0,49.246876,-123.181981,Dunbar-Southlands,00:00:00,Theft from Vehicle,NaT,NaT
8,0,2015-01-01,3,2015,1,1,1,0,1,446302,...,NaN,0,0,49.224975,-123.149487,Kerrisdale,15:00:00,Theft from Vehicle,NaT,NaT
9,0,2015-01-01,3,2015,1,1,1,0,1,446752,...,NaN,0,0,49.283559,-123.114439,Central Business District,17:20:00,Break and Enter Commercial,NaT,NaT


In [71]:
# Add Location
location = pd.read_csv('datasets/final/location.csv')
df['NEIGHBORHOOD'] = df.NEIGHBORHOOD.str.strip()
df['NEIGHBORHOOD'] = df.NEIGHBORHOOD.str.lower()
df['NEIGHBORHOOD'] = df.NEIGHBORHOOD.str.replace('-', ' ')
df['NEIGHBORHOOD'] = df.NEIGHBORHOOD.str.replace('   ', ' ')

df = pd.merge(df, location, left_on=['ADDRESS', 'LAT', 'LON', 'NEIGHBORHOOD'], right_on=['ADDRESS', 'LAT', 'LON', 'NEIGHBORHOOD'], how='left')
display(df)

,Date_key,Date,Day_of_week,Year,Month,Day,Week,Weekend,season,Crime_key,...,AGE_20_TO_29,AGE_30_TO_39,AGE_40_TO_49,AGE_50_TO_59,AGE_60_OVER,MEDIAN_AGE_ALL,MEDIAN_AGE_MALE,MEDIAN_AGE_FEMALE,PER_CAPITA_INCOME,PCT_LOW_INCOME
0,0,2015-01-01,3,2015,1,1,1,0,1,440785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2015-01-01,3,2015,1,1,1,0,1,440797,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,2015-01-01,3,2015,1,1,1,0,1,440798,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,2015-01-01,3,2015,1,1,1,0,1,440802,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2015-01-01,3,2015,1,1,1,0,1,441402,...,5115.0,4780.0,4915.0,5290.0,8095.0,42.100000,41.000000,43.100000,27255.0,16.900000
5,0,2015-01-01,3,2015,1,1,1,0,1,442639,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,2015-01-01,3,2015,1,1,1,0,1,443601,...,5115.0,4780.0,4915.0,5290.0,8095.0,42.100000,41.000000,43.100000,27255.0,16.900000
7,0,2015-01-01,3,2015,1,1,1,0,1,443739,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,2015-01-01,3,2015,1,1,1,0,1,446302,...,2055.0,1105.0,2015.0,2290.0,3585.0,45.600000,44.200000,46.400000,35064.0,22.000000
9,0,2015-01-01,3,2015,1,1,1,0,1,446752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# Add Weather
weather = pd.read_csv('datasets/weather.csv')
weather['Date'] = pd.to_datetime(weather['Date'])
df = pd.merge(df, weather, left_on=['City', 'Date'], right_on=['City', 'Date'], how='left')
display(df)

,Date_key,Date,Day_of_week,Year,Month,Day,Week,Weekend,season,Crime_key,...,MEDIAN_AGE_ALL,MEDIAN_AGE_MALE,MEDIAN_AGE_FEMALE,PER_CAPITA_INCOME,PCT_LOW_INCOME,Weather_key,Temperature,Weather,Sunrise,Sunset
0,0,2015-01-01,3,2015,1,1,1,0,1,440785,...,NaN,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46
1,0,2015-01-01,3,2015,1,1,1,0,1,440797,...,NaN,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46
2,0,2015-01-01,3,2015,1,1,1,0,1,440798,...,NaN,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46
3,0,2015-01-01,3,2015,1,1,1,0,1,440802,...,NaN,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46
4,0,2015-01-01,3,2015,1,1,1,0,1,441402,...,42.100000,41.000000,43.100000,27255.0,16.900000,1644.0,273.628384,sky is clear,07:21,16:46
5,0,2015-01-01,3,2015,1,1,1,0,1,442639,...,NaN,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46
6,0,2015-01-01,3,2015,1,1,1,0,1,443601,...,42.100000,41.000000,43.100000,27255.0,16.900000,1644.0,273.628384,sky is clear,07:21,16:46
7,0,2015-01-01,3,2015,1,1,1,0,1,443739,...,NaN,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46
8,0,2015-01-01,3,2015,1,1,1,0,1,446302,...,45.600000,44.200000,46.400000,35064.0,22.000000,1644.0,273.628384,sky is clear,07:21,16:46
9,0,2015-01-01,3,2015,1,1,1,0,1,446752,...,NaN,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46


In [73]:
# Add is_nighttime
df['Is_Nighttime'] = 1
df.loc[(df.TIME > df.Sunrise) & (df.TIME < df.Sunset), 'Is_Nighttime'] = 0
display(df)

,Date_key,Date,Day_of_week,Year,Month,Day,Week,Weekend,season,Crime_key,...,MEDIAN_AGE_MALE,MEDIAN_AGE_FEMALE,PER_CAPITA_INCOME,PCT_LOW_INCOME,Weather_key,Temperature,Weather,Sunrise,Sunset,Is_Nighttime
0,0,2015-01-01,3,2015,1,1,1,0,1,440785,...,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46,1
1,0,2015-01-01,3,2015,1,1,1,0,1,440797,...,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46,1
2,0,2015-01-01,3,2015,1,1,1,0,1,440798,...,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46,1
3,0,2015-01-01,3,2015,1,1,1,0,1,440802,...,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46,0
4,0,2015-01-01,3,2015,1,1,1,0,1,441402,...,41.000000,43.100000,27255.0,16.900000,1644.0,273.628384,sky is clear,07:21,16:46,1
5,0,2015-01-01,3,2015,1,1,1,0,1,442639,...,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46,1
6,0,2015-01-01,3,2015,1,1,1,0,1,443601,...,41.000000,43.100000,27255.0,16.900000,1644.0,273.628384,sky is clear,07:21,16:46,1
7,0,2015-01-01,3,2015,1,1,1,0,1,443739,...,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46,1
8,0,2015-01-01,3,2015,1,1,1,0,1,446302,...,44.200000,46.400000,35064.0,22.000000,1644.0,273.628384,sky is clear,07:21,16:46,0
9,0,2015-01-01,3,2015,1,1,1,0,1,446752,...,NaN,NaN,NaN,NaN,1644.0,273.628384,sky is clear,07:21,16:46,1


In [74]:
# Add stock data
stock = pd.read_csv('datasets/StockData.csv')
stock['date'] = pd.to_datetime(stock.date)
df = pd.merge(df, stock, left_on='Date', right_on='date', how='left')
display(df)

,Date_key,Date,Day_of_week,Year,Month,Day,Week,Weekend,season,Crime_key,...,Sunrise,Sunset,Is_Nighttime,stockKey,date,changeOpenClose,changeLastDay,changeLastWeek,changeLastMonth,changeLastYear
0,0,2015-01-01,3,2015,1,1,1,0,1,440785,...,07:21,16:46,1,1095,2015-01-01,0.613481,0.0,-0.618917,-0.182061,8.357103
1,0,2015-01-01,3,2015,1,1,1,0,1,440797,...,07:21,16:46,1,1095,2015-01-01,0.613481,0.0,-0.618917,-0.182061,8.357103
2,0,2015-01-01,3,2015,1,1,1,0,1,440798,...,07:21,16:46,1,1095,2015-01-01,0.613481,0.0,-0.618917,-0.182061,8.357103
3,0,2015-01-01,3,2015,1,1,1,0,1,440802,...,07:21,16:46,0,1095,2015-01-01,0.613481,0.0,-0.618917,-0.182061,8.357103
4,0,2015-01-01,3,2015,1,1,1,0,1,441402,...,07:21,16:46,1,1095,2015-01-01,0.613481,0.0,-0.618917,-0.182061,8.357103
5,0,2015-01-01,3,2015,1,1,1,0,1,442639,...,07:21,16:46,1,1095,2015-01-01,0.613481,0.0,-0.618917,-0.182061,8.357103
6,0,2015-01-01,3,2015,1,1,1,0,1,443601,...,07:21,16:46,1,1095,2015-01-01,0.613481,0.0,-0.618917,-0.182061,8.357103
7,0,2015-01-01,3,2015,1,1,1,0,1,443739,...,07:21,16:46,1,1095,2015-01-01,0.613481,0.0,-0.618917,-0.182061,8.357103
8,0,2015-01-01,3,2015,1,1,1,0,1,446302,...,07:21,16:46,0,1095,2015-01-01,0.613481,0.0,-0.618917,-0.182061,8.357103
9,0,2015-01-01,3,2015,1,1,1,0,1,446752,...,07:21,16:46,1,1095,2015-01-01,0.613481,0.0,-0.618917,-0.182061,8.357103


In [75]:
# Add Events
events = pd.read_csv('datasets/events_mapping.csv')
events['date'] = pd.to_datetime(events.date)
df = pd.merge(df, events, left_on='Date', right_on='date', how='left')
display(df)

,Date_key,Date,Day_of_week,Year,Month,Day,Week,Weekend,season,Crime_key,...,city,date_y,endDate,endTime,eventKey,isOffWork,name,startDate,startTime,type
0,0,2015-01-01,3,2015,1,1,1,0,1,440785,...,Vancouver,2015-01-01,2015-01-01,NaN,37.0,1.0,Hockey,2015-01-01,22:00:00,Sport
1,0,2015-01-01,3,2015,1,1,1,0,1,440785,...,Vancouver,2015-01-01,2015-01-01,NaN,38.0,1.0,New Year's Day,2015-01-01,NaN,Public Holiday
2,0,2015-01-01,3,2015,1,1,1,0,1,440785,...,Denver,2015-01-01,2015-01-01,NaN,39.0,1.0,New Year's Day,2015-01-01,NaN,Public Holiday
3,0,2015-01-01,3,2015,1,1,1,0,1,440797,...,Vancouver,2015-01-01,2015-01-01,NaN,37.0,1.0,Hockey,2015-01-01,22:00:00,Sport
4,0,2015-01-01,3,2015,1,1,1,0,1,440797,...,Vancouver,2015-01-01,2015-01-01,NaN,38.0,1.0,New Year's Day,2015-01-01,NaN,Public Holiday
5,0,2015-01-01,3,2015,1,1,1,0,1,440797,...,Denver,2015-01-01,2015-01-01,NaN,39.0,1.0,New Year's Day,2015-01-01,NaN,Public Holiday
6,0,2015-01-01,3,2015,1,1,1,0,1,440798,...,Vancouver,2015-01-01,2015-01-01,NaN,37.0,1.0,Hockey,2015-01-01,22:00:00,Sport
7,0,2015-01-01,3,2015,1,1,1,0,1,440798,...,Vancouver,2015-01-01,2015-01-01,NaN,38.0,1.0,New Year's Day,2015-01-01,NaN,Public Holiday
8,0,2015-01-01,3,2015,1,1,1,0,1,440798,...,Denver,2015-01-01,2015-01-01,NaN,39.0,1.0,New Year's Day,2015-01-01,NaN,Public Holiday
9,0,2015-01-01,3,2015,1,1,1,0,1,440802,...,Vancouver,2015-01-01,2015-01-01,NaN,37.0,1.0,Hockey,2015-01-01,22:00:00,Sport


In [76]:
final = df[['Date_key', 'eventKey', 'Crime_key', 'Location_key', 'Weather_key', 'stockKey', 'IS_FATAL', 'IS_TRAFFIC', 'Is_Nighttime']]
display(final)
final.to_csv('datasets/final/fact.csv', index=False)

,Date_key,eventKey,Crime_key,Location_key,Weather_key,stockKey,IS_FATAL,IS_TRAFFIC,Is_Nighttime
0,0,37.0,440785,NaN,1644.0,1095,0,0,1
1,0,38.0,440785,NaN,1644.0,1095,0,0,1
2,0,39.0,440785,NaN,1644.0,1095,0,0,1
3,0,37.0,440797,45411.0,1644.0,1095,0,0,1
4,0,38.0,440797,45411.0,1644.0,1095,0,0,1
5,0,39.0,440797,45411.0,1644.0,1095,0,0,1
6,0,37.0,440798,45411.0,1644.0,1095,0,0,1
7,0,38.0,440798,45411.0,1644.0,1095,0,0,1
8,0,39.0,440798,45411.0,1644.0,1095,0,0,1
9,0,37.0,440802,6929.0,1644.0,1095,0,0,0
